In [69]:
!git clone https://github.com/amnatasneem/machine-learning-fall-2023-final-project.git

fatal: destination path 'machine-learning-fall-2023-final-project' already exists and is not an empty directory.


In [70]:
!pip install scikeras

In [71]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.ensemble import RandomForestRegressor

In [72]:
heart_data = pd.read_csv('/content/machine-learning-fall-2023-final-project/data/heart.csv', sep=",")
o2_data = pd.read_csv("/content/machine-learning-fall-2023-final-project/data/o2Saturation.csv")

In [73]:
heart_data.head(10)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


In [74]:
o2_data.head(10)

,98.6
0,98.6
1,98.6
2,98.6
3,98.1
4,97.5
5,97.5
6,97.5
7,97.5
8,97.5
9,97.5


In [75]:
# Classification: heart attack prediction
# Target column index = 13 ("output")
X_classification = heart_data.drop(heart_data.columns[13], axis=1)
y_classification = heart_data[heart_data.columns[13]]

# Split the data (train and test)
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_classification, y_classification, test_size=0.2, random_state=50
)

In [76]:
# Standardize features
scaler = StandardScaler()
X_train_class_scaled = scaler.fit_transform(X_train_class)
X_test_class_scaled = scaler.transform(X_test_class)

In [77]:
# Logistic regression
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_class_scaled, y_train_class)
y_pred_labels = log_reg_model.predict(X_test_class_scaled)

In [78]:
# Print classification results
print("Logistic Regression - Classification Results: ")
print(classification_report(y_test_class, y_pred_labels))

Logistic Regression - Classification Results: 
              precision    recall  f1-score   support

           0       0.80      0.64      0.71        25
           1       0.78      0.89      0.83        36

    accuracy                           0.79        61
   macro avg       0.79      0.76      0.77        61
weighted avg       0.79      0.79      0.78        61



In [79]:
# Neural network
def create_nn_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train_class_scaled.shape[1], activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

nn_model = KerasClassifier(build_fn=create_nn_model, epochs=10, batch_size=32, verbose=0)
nn_model.fit(X_train_class_scaled, y_train_class)
y_pred_nn = nn_model.predict(X_test_class_scaled)

# Print neural network classification results
print("Neural Network - Classification Results:")
print(confusion_matrix(y_test_class, y_pred_nn))
print(classification_report(y_test_class, y_pred_nn))

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Neural Network - Classification Results:
[[16  9]
 [11 25]]
              precision    recall  f1-score   support

           0       0.59      0.64      0.62        25
           1       0.74      0.69      0.71        36

    accuracy                           0.67        61
   macro avg       0.66      0.67      0.66        61
weighted avg       0.68      0.67      0.67        61



In [80]:
import numpy as np

# Regression: O2 saturation (X) and if a heart attack occurred (y)
extra = o2_data.drop(o2_data.columns[0], axis=1)

# Convert the Series to a NumPy array and reshape
X_regression = o2_data[o2_data.columns[0]].head(303).values.reshape(-1, 1)

y_regression = heart_data[heart_data.columns[13]]

In [81]:
# Split the data (train and test)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=50
)

In [82]:
# Standardize features
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

In [83]:
# For regression, we will try out three different models:
# 2d polynomial regression, neural network regression, and random forest regression

In [84]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 2D Polynomial Regression
poly = PolynomialFeatures(degree=2)
X_train_reg_poly = poly.fit_transform(X_train_reg_scaled)
X_test_reg_poly = poly.transform(X_test_reg_scaled)

# Train linear regression on polynomial features
poly_reg_model = LinearRegression().fit(X_train_reg_poly, y_train_reg)
y_pred_reg_poly = poly_reg_model.predict(X_test_reg_poly)

# Print regression results for Polynomial Regression
print("\n2D Polynomial Regression - Results:")
print("Residual sum of squares:", mean_squared_error(y_test_reg, y_pred_reg_poly))
# Explained variance score: 1 is perfect prediction
print("Variance score:", poly_reg_model.score(X_test_reg_poly, y_test_reg))

y_predicted = poly_reg_model.predict(X_test_reg_poly)
R_square = r2_score(y_predicted, y_test_reg)
print('Coefficient of Determination', R_square)


2D Polynomial Regression - Results:
Residual sum of squares: 0.21266613218884292
Variance score: 0.12074369125035056
Coefficient of Determination -6.396215461825425


In [85]:
# Interpretation:
# Residual Sum of Squares: low is good - model is making predictions close to the actual values.
# Variance Score (R-squared): low - suggests that model may not be explaining much of the variance in the target variable.
# Coefficient of Determination: negative is unusual - not capturing the pattern in the data at all.
# Come back to this model

In [86]:
# Neural network regression
def create_reg_nn_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train_reg.shape[1], activation="relu"))
    model.add(Dense(1, activation="linear"))
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model

nn_reg_model = create_reg_nn_model()
nn_reg_model.fit(X_train_reg, y_train_reg, epochs=10, batch_size=32, verbose=0)
y_pred_reg_nn = nn_reg_model.predict(X_test_reg)

# Print regression results for Neural Network
print("\nNeural Network Regression - Results:")
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg_nn))

2/2 [==============================] - 0s 6ms/step

Neural Network Regression - Results:
Mean Squared Error: 29.336143813400557
